# Azure Data University: mlos_bench SQLite data analysis (Student's workbook)

In this notebook, we look at the data from the `mlos_bench` experiment we run in class to find a better SQLite configuration.

### 1. Data collection

Before using this notebook, run the following commands in the integrated terminal of this codespace:

```sh
conda activate mlos

mlos_bench --config "./config/cli/local-sqlite-opt.jsonc" --globals "./config/experiments/sqlite-sync-journal-pagesize-caching-experiment.jsonc" --max-iterations 100
```

> See Also: [README.md](./README.md) for further instructions.

After letting it run for a few trials (it should take 10 to 15 minutes), we can start analyzing the autotuning data produced by the `mlos_bench` framework.

### 2. Import packages

First, we import a few common third-party libraries for data analysis.

In [ ]:
# Import a few popular third-party packages for data analysis. They come pre-installed with Anaconda.
import pandas
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Import mlos_bench Storage API to access the experimental data.
from mlos_bench.storage import from_config

In [ ]:
# Cosmetic: Suppress some annoying warnings from third-party data visualization packages
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### 3. Connect to the DB using existing mlos_bench configs

We reuse the existing mlos_bench framework configuration file that contains the DB connection parameters. This way we make sure to connect to the same database that our framework uses to store the experimental data.

In [ ]:
storage = from_config(config_file="storage/sqlite.jsonc")

### 4. Load the data for our experiment

In [ ]:
storage.experiments

You should see a record for our experiment in the DB. Let's look at the data associated with it.

In [ ]:
experiment_id = "sqlite-opt-demo"

### 5. Get all data for one experiment

In [ ]:
exp = storage.experiments[experiment_id]
exp

Main method that combines the information about each trial along with the trial configuration parameters and its results, is the property `.results`. It conveniently returns all data about the experiment is a single Pandas DataFrame.

In [ ]:
# View some of the result data associated with that experiment.
df = exp.results
df.head()

Each record of the DataFrame has the information about the trial, e.g., its timestamp and status, along with the configuration parameters (columns prefixed with `config.`) and the benchmark results (columns prefixed with `result.`). Let's look at the first record to see all these fields.

In [ ]:
df.loc[1]

### 6. Plot the results

#### 6.1. Plot the behavior of the optimizer

In [ ]:
# TODO:
# 1. plot of the optimizer's best config found so far (convergence rate of the optimizer)
# 2. plot of the performance of each config the optimizer found (search behavior of the optimizer)

#### 6.2. Plot the results of one metric vs. another for some tunable

The intent is to explore parameter importance and impact on different metrics (both application performance and system resource usage).

In [ ]:
# Categorical tunable to consider:
CATEGORY = "config.synchronous"

# A system resource metric to analyze.
METRIC = "result.File system outputs"

# Which performance metric to plot on the Y-axis.
SCORE = "result.90th Percentile Latency (microseconds)"

In [ ]:
plt.rcParams["figure.figsize"] = (6, 5)

sns.scatterplot(data=df, x=METRIC, y=SCORE, hue=CATEGORY, marker='o', alpha=0.6)

plt.title("Experiment: " + exp.exp_id)
plt.grid()
plt.show()

The results are here, but the outliers make it really difficult to understand what's going on. Let's switch to the log scale and see if that helps.

#### 6.3. Plot on log scale

In [ ]:
plt.rcParams["figure.figsize"] = (6, 5)

sns.scatterplot(data=df, x=METRIC, y=SCORE, hue=CATEGORY, marker='o', alpha=0.6)

plt.xscale('log')
plt.yscale('log')

plt.title("Experiment: " + exp.exp_id)
plt.grid()
plt.show()

 Now we can see that setting `synchronous=off` seems to seems to improve the latency a lot. Apparently, the optimizer had also noticed that and focused on exploring a particular area of the configuration space. Let's switch back to the linear scale and zoom in to that region.

#### 6.4. Zoom in

In [ ]:
plt.rcParams["figure.figsize"] = (6, 4)

sns.scatterplot(data=df, x=METRIC, y=SCORE, hue=CATEGORY, marker='o', alpha=0.7)

plt.xlim(100000, 350000)
plt.ylim(950, 1250)

plt.title("Experiment: " + exp.exp_id)
plt.grid()
plt.show()

The latency seems to be minimal when the configuration parameter `synchronous` is set to `off` and when the metric `File system outputs` is in range of 100..300K. Let's focus on that subset of data and see what other configuration settings get us there.

#### 6.5. Look at other configuration parameters

In [ ]:
df_lim = df[(df[CATEGORY] == "off") & (df[METRIC] > 100000) & (df[METRIC] < 300000)]

In [ ]:
plt.rcParams["figure.figsize"] = (6, 4)

sns.scatterplot(data=df_lim, x="config.cache_size", y=SCORE, hue="config.journal_mode", marker='o', alpha=0.7)

plt.title("Experiment: " + exp.exp_id)
plt.grid()
plt.show()

Again, we see that setting `journal_mode` to `wal` and `cache_size` to the value between 500MB and 2GB seem to produce good results, but we need more experiments to explore that hypothesis.

### 7. Outro

If you feel curious, please go ahead and play with the SQLite data in the cells below.

After that, please open other notebooks in this repository and explore the data you have collected in this class as well as the results from our MySQL optimization experiments:

* [**mlos_demo_sqlite_teachers.ipynb**](mlos_demo_sqlite_teachers.ipynb) - Teacher's copy, don't peek! :-) Here we analyze the data from 100 trials of SQLite optimization we ran in this codespace before the class. The results you get in the workshop should look similar.
* [**mlos_demo_mysql.ipynb**](mlos_demo_mysql.ipynb) - Look at the actual production data we've collected in serveral experiment for MySQL Server optimization on Azure.